In [1]:
### Parameters
tol = 1e-5
balance_strength = 2
save = False
# Number of spatial orbitals in a block
block_size = 4
# Number of electrons per block
ne_per_block = 4
# +- difference in number of electrons per block
ne_range = 2
# Number of killer operators for each CAS block
n_killer = 3
# Running Full CI to check compute the ground state, takes exponentially amount of time to execute
FCI = False
# Checking symmetries of the planted Hamiltonian, very costly
check_symmetry = False
# Concatenate the states in each block to compute the ground state solution
check_state = True
# File path and name
fcidump_path ="fcidumps_catalysts"
fcidump_filename = "fcidump.2_co2_6-311++G__"
fcidump_output_path = "fcidumps_catalysts_planted_solutions"

In [2]:
"""Construct CAS Hamiltonians with cropping functions
"""
import CAS_Cropping.saveload_utils as sl
import CAS_Cropping.ferm_utils as feru
import CAS_Cropping.csa_utils as csau
import CAS_Cropping.var_utils as varu
import openfermion as of
import numpy as np
from CAS_Cropping.sdstate import *
from itertools import product
import random
import h5py
import sys
import pickle
import os
from pathlib import Path
import CAS.dmrghandler.src.dmrghandler.pyscf_wrappers
import CAS.dmrghandler.src.dmrghandler.dmrg_calc_prepare
import CAS.dmrghandler.src.dmrghandler.qchem_dmrg_calc
import CAS.dmrghandler.src.dmrghandler as dmrghandler
from CAS_Cropping.matrix_utils import construct_orthogonal
import pyscf.tools.fcidump
import sys
sys.path.append('./')


def construct_blocks(b: int, spin_orbs: int, spin_orb = False):
    """Construct CAS blocks of size b for spin_orbs/spatial number of orbitals"""
    if spin_orb:
        b = b * 2
    k = []
    tmp = [0]
    for i in range(1, spin_orbs):
        if i % b == 0:
            k.append(tmp)
            tmp = [i]
        else:
            tmp.append(i)
    if len(tmp) != 0:
        k.append(tmp)
    return k
        
def get_truncated_cas_tbt(H, k, casnum):
#     Trunctate the original Hamiltonian two body tensor into the cas block structures
    Hobt, Htbt = H
    n = Htbt.shape[0]
    cas_tbt = np.zeros([n, n, n, n])
    cas_obt = np.zeros([n, n])
    cas_x = np.zeros(casnum)
    idx = 0
    for block in k:
        for p, q in product(block, repeat = 2):
            cas_obt[p,q] = Hobt[p,q]
            cas_x[idx] = Hobt[p,q]
            idx += 1
        for a, b, c, d in product(block, repeat = 4):
            cas_tbt [a,b,c,d] = Htbt [a,b,c,d]
            cas_x[idx] = Htbt[a,b,c,d]
            idx += 1
    return cas_obt, cas_tbt, cas_x

def in_orbs(term, orbs):
    """Return if the term is a local excitation operator within orbs"""
    if len(term) == 2:
        return term[0][0] in orbs and term[1][0] in orbs
    elif len(term) == 4:
        return term[0][0] in orbs and term[1][0] in orbs and term[2][0] in orbs and term[3][0] in orbs
    return False

def transform_orbs(term, orbs):
    """Transform the operator term to align the orbs starting from 0"""
#     pass
    if len(term) == 2:
        return ((orbs.index(term[0][0]), 1), (orbs.index(term[1][0]), 0))
    if len(term) == 4:
        return ((orbs.index(term[0][0]), 1), (orbs.index(term[1][0]), 0), 
               (orbs.index(term[2][0]), 1), (orbs.index(term[3][0]), 0))   
    return None

def solve_enums(H, k, ne_per_block = 0, ne_range = 0, balance_t = 10):
    """Solve for number of electrons in each CAS block with FCI within the block,
    H = (obt, tbt) as the Hamiltonian in spatial orbi
    """ 
    cas_obt = H[0]
    cas_tbt = H[1]
#     print(cas_obt.shape)
#     print(cas_tbt.shape)

    e_nums = []
    states = []
    E_cas = 0
    for orbs in k:
        s = orbs[0]
        t = orbs[-1] + 1
        norbs = len(orbs)
        ne = min(ne_per_block + random.randint(-ne_range, ne_range), norbs * 2 - 1)
        print(f"Ne within current block: {ne}")
#         Construct (Ne^-ne)^2 terms in matrix, to enforce structure of states
        if ne_per_block != 0:
            balance_obt = np.zeros([norbs, norbs])
            balance_tbt = np.zeros([norbs, norbs,  norbs, norbs])
            for p, q in product(range(norbs), repeat = 2):
                balance_tbt[p,p,q,q] += 1
            for p in range(len(orbs)):
                balance_obt[p,p] -= 2 * ne
#             Construct 2e tensor to enforce the Ne in the ground state.
            strength = balance_t * (1 + random.random())
#             tmp_tbt = np.add(tmp_tbt, balance_tbt)
        flag = True
        while flag:
            strength *= 2
            cas_tbt[s:t, s:t, s:t, s:t] = np.add(cas_tbt[s:t, s:t, s:t, s:t], strength * balance_tbt)
            cas_obt[s:t, s:t] = np.add(cas_obt[s:t, s:t], strength * balance_obt)
#             Set spin_orb to False to represent spatial orbital basis Hamiltonian
            tmp = feru.get_ferm_op(cas_tbt[s:t, s:t, s:t, s:t], spin_orb = False)
            tmp += feru.get_ferm_op(cas_obt[s:t, s:t], spin_orb = False)
            sparse_H_tmp = of.get_sparse_operator(tmp)
            tmp_E_min, t_sol = of.get_ground_state(sparse_H_tmp)
            st = sdstate(n_qubit = len(orbs) * 2) 
            for i in range(len(t_sol)):
                if np.linalg.norm(t_sol[i]) > np.finfo(np.float32).eps:
                    st += sdstate(s = i, coeff = t_sol[i])
#             print(f"state norm: {st.norm()}")
            st.normalize()
            E_st = st.exp(tmp)
            flag = False
            for sd in st.dic:
                ne_computed = bin(sd)[2:].count('1')
                if ne_computed != ne:
                    print("Not enough balance, adding more terms")
                    print(bin(sd))
                    flag = True
                    break
        print(f"E_min: {tmp_E_min} for orbs: {orbs}")
        print(f"current state Energy: {E_st}")
        E_cas += E_st
        states.append(st)
        e_nums.append(ne)                
    return e_nums, states, E_cas

# Killer Construction
def construct_killer(k, e_num, n = 0, const = 1e-2, t = 1e2, n_killer = 3):
    """ Construct a killer operator for CAS Hamiltonian, based on cas block structure of k and the size of killer is 
    given in k, the number of electrons in each CAS block of the ground state
    is specified by e_nums. t is the strength of quadratic balancing terms for the killer with respect to k,
    n_killer specifies the number of operators O to choose.
    """
    if not n:
        n = max([max(orbs) for orbs in k])
    killer = of.FermionOperator.zero()
    for i in range(len(k)):
        orbs = k[i]
        outside_orbs = [j for j in range(n) if j not in orbs]
    #     Construct Ne
        Ne = sum([of.FermionOperator("{}^ {}".format(i, i)) for i in orbs])
    #     Construct O, for O as combination of Epq which preserves Sz and S2
        if len(outside_orbs) >= 4:
            tmp = 0
            while tmp < n_killer:
                p, q = random.sample(outside_orbs, 2)
                if abs(p - q) > 1:
#                     Constructing symmetry conserved killers
                    O = of.FermionOperator.zero()
#                     if p % 2 != 0:
#                         p -= 1
#                     if q % 2 != 0:
#                         q -= 1
#                     ferm_op = of.FermionOperator("{}^ {}".format(p, q)) + of.FermionOperator("{}^ {}".format(q, p))
#                     O += ferm_op
#                     O += of.hermitian_conjugated(ferm_op)
#                     ferm_op = of.FermionOperator("{}^ {}".format(p + 1, q + 1)) + of.FermionOperator("{}^ {}".format(q + 1, p + 1))
#                     O += ferm_op
#                     O += of.hermitian_conjugated(ferm_op)
                    ferm_op = of.FermionOperator("{}^ {}".format(p, q)) + of.FermionOperator("{}^ {}".format(q, p))
                    O += ferm_op
                    O += of.hermitian_conjugated(ferm_op)
                    k_const = const * (1 + np.random.rand())
                    killer += k_const * O * (Ne - e_nums[i])
                    tmp += 1
        killer += t * (1 + np.random.rand()) * const * ((Ne - e_nums[i]) ** 2)
    killer_obt = feru.get_obt(killer, n = n, spin_orb = True)
    killer_tbt = feru.get_two_body_tensor(killer, n = n)
#     Killer constant term
    c = killer.terms[()]
    return c, killer_obt, killer_tbt

def construct_orbs(key: str):
#     Contruct k from the given key
    count = 0
    lis = key.split("-")
    k = []
    for i in lis:
        tmp = int(i)
        k.append(list(range(count, count + tmp)))
        count += tmp
    return k

def get_param_num(n, k, complex = False):
    '''
    Counting the parameters needed, where k is the number of orbitals occupied by CAS Fragments,
    and n-k orbitals are occupied by the CSA Fragments
    '''
    if not complex:
        upnum = int(n * (n - 1) / 2)
    else:
        upnum = n * (n - 1)
    casnum = 0
    for block in k:
        casnum += len(block) ** 4 + len(block) ** 2
    pnum = upnum + casnum
    return upnum, casnum, pnum

def construct_Hamiltonian_with_solution(path, file_name):
    with open(ps_path + f_name, 'rb') as handle:
        dic = pickle.load(handle)
        key = list(dic.keys())[0]
        dic = dic[key]
    cas_x = dic["cas_x"]
    killer = dic["killer"]
    killer_c = killer[0]
    k_obt = killer[1]
    k_tbt = killer[2]
    k = dic["k"]
    upnum = dic["upnum"]
    spatial_orbs = dic["spatial_orbs"]
#     CAS 2e tensor
    obt, tbt = get_cas_matrix(cas_x, spatial_orbs, k)
    H_cas = (0, obt, tbt)
    H_with_killer = (killer_c, obt+k_obt, tbt+k_tbt)
#     Set up random unitary to hide 2e tensor
    random_uparams = np.random.rand(upnum)
    U = construct_orthogonal(spatial_orbs, random_uparams)
#     Hide 2e etensor with random unitary transformation
    H_hidden = (0, orbtransf(obt, U), orbtransf(tbt, U))
    H_killer_hidden = (killer_c, orbtransf(H_with_killer[1], U), orbtransf(H_with_killer[2], U))
    return H_cas, H_hidden, H_with_killer, H_killer_hidden

In [3]:
def check_for_incorrect_spin_terms(tbt_to_check):
    num_incorrect_terms = 0
    #Check no incorrect spin terms present
    num_spin_orbitals = tbt_to_check.shape[0]
    no_incorrect_terms = True
    for piter in range(num_spin_orbitals):
        for qiter in range(num_spin_orbitals):
            for riter in range(num_spin_orbitals):
                for siter in range(num_spin_orbitals):
                    if piter % 2 == 0 and qiter % 2 == 0 and riter % 2 == 0 and siter % 2 == 0:
                        continue
                    if piter % 2 == 1 and qiter % 2 == 1 and riter % 2 == 1 and siter % 2 == 1:
                        continue
                    if piter % 2 == 0 and qiter % 2 == 0 and riter % 2 == 1 and siter % 2 == 1:
                        continue
                    if piter % 2 == 1 and qiter % 2 == 1 and riter % 2 == 0 and siter % 2 == 0:
                        continue
                    if not np.isclose(tbt_to_check[piter, qiter, riter, siter], 0.0):
                        # print(f"Incorrect spin term present in two body tensor at indices {piter}, {qiter}, {riter}, {siter}: {tbt_to_check[piter, qiter, riter, siter]}")
                        no_incorrect_terms = False
                        num_incorrect_terms += 1

    return no_incorrect_terms, num_incorrect_terms

def check_tbt(tbt_to_check):
    num_spin_orbitals = tbt_to_check.shape[0]
    dummy_obt = np.zeros((num_spin_orbitals, num_spin_orbitals))
    no_incorrect_terms, num_incorrect_terms = check_for_incorrect_spin_terms(tbt_to_check)
    print(f"No incorrect spin terms present: {no_incorrect_terms}")
    print(f"Number of incorrect terms: {num_incorrect_terms}")

    spin_symm_check_passed = dmrghandler.dmrg_calc_prepare.check_spin_symmetry(one_body_tensor=dummy_obt, two_body_tensor=tbt_to_check)
    print(f"Spin symmetry check passed: {spin_symm_check_passed}")

    permutation_symmetries_complex_orbitals_check_passed = dmrghandler.dmrg_calc_prepare.check_permutation_symmetries_complex_orbitals(
        dummy_obt, tbt_to_check
    )
    print(f"Permutation symmetries complex orbitals check passed: {permutation_symmetries_complex_orbitals_check_passed}")

    permutation_symmetries_real_orbitals_check_passed = dmrghandler.dmrg_calc_prepare.check_permutation_symmetries_real_orbitals(
        dummy_obt, tbt_to_check
    )
    print(f"Permutation symmetries real orbitals check passed: {permutation_symmetries_real_orbitals_check_passed}")

In [4]:
"""
Tensor Conversion functions
"""
def chem_to_physicist_notation_spin_orbitals(obt, tbt_local):
    # Assume tbt_local is defined such that
    # H = γ_PQRS a†_P a_Q a†_R a_S
    # PQRS are spin-orbitals

    # Output tensors assume H = H_PQ a†_P a_Q + 0.5*G_PQRS a†_P a†_R a_S a_Q

    num_spin_orbitals = tbt_local.shape[0]
    num_ortbitals = num_spin_orbitals // 2

    G_PQRS = 2*tbt_local.copy()

    H_PQ = np.zeros((num_spin_orbitals, num_spin_orbitals))
    for p in range(num_spin_orbitals):
        for q in range(num_spin_orbitals):
            for r in range(num_spin_orbitals):
                H_PQ[p, q] += 0.5*G_PQRS[p, r, r, q]

    return H_PQ + obt, G_PQRS

def chem_spin_orb_to_phys_spatial_orb(tbt_local):
    # Assume tbt_local is defined such that
    # H = γ_PQRS a†_P a_Q a†_R a_S
    # PQRS are spin-orbitals

    # Output tensors assume H = H_ij a†_i a_j + 0.5*G_ijkl a†_i a†_k a_l a_j, 
    # `ijkl` refer to *spatial* orbitals


    #First, convert to physicist notation with spin-orbitals
    H_PQ, G_PQRS = chem_to_physicist_notation_spin_orbitals(tbt_local)

    
    (H_ij, 
    G_ijkl, 
    spin_symm_broken) = dmrghandler.dmrg_calc_prepare.spinorbitals_to_orbitals(one_body_tensor=H_PQ, 
                                                                                two_body_tensor=G_PQRS)
    print(f"Spin symmetry broken: {spin_symm_broken}")                                                                                
    return H_ij, G_ijkl

In [5]:
(
one_body_tensor,
two_body_tensor,
nuc_rep_energy,
num_orbitals,
num_spin_orbitals,
num_electrons,
two_S,
two_Sz,
orb_sym,
extra_attributes,
) = dmrghandler.dmrg_calc_prepare.load_tensors_from_fcidump(data_file_path=Path(fcidump_path)/Path(fcidump_filename), molpro_orbsym_convention=True)

spin_orbs = 2*num_orbitals
spatial_orbs = num_orbitals

print(f"Number of spin orbitals: {spin_orbs}")

h_ij = one_body_tensor
g_ijkl = two_body_tensor
# Hamiltonian assumed is `H = E_0 + h_ij a†_i a_j + 0.5*g_ijkl a†_i a†_k a_l a_j`, `ijkl` refer to *spatial* orbitals

Parsing fcidumps_catalysts/fcidump.2_co2_6-311++G__
Number of spin orbitals: 12


In [6]:
spatial_orbs = h_ij.shape[0]
print(f"Number of spatial orbitals: {spatial_orbs}")
#     spatial_orbs = spin_orbs // 2
#     onebody_tbt = feru.onebody_to_twobody(one_body)
#     Htbt = np.add(two_body, onebody_tbt)
h_ij -= 0.5*np.einsum("prrq->pq",g_ijkl)
g_ijkl *= 0.5
H = (h_ij, g_ijkl)
k = construct_blocks(block_size, spatial_orbs)
print(f"orbital splliting: {k}")
upnum, casnum, pnum = get_param_num(spatial_orbs, k, complex = False)
cas_obt, cas_tbt, cas_x = get_truncated_cas_tbt(H, k, casnum)
H_cas = [cas_obt, cas_tbt]

Number of spatial orbitals: 6
orbital splliting: [[0, 1, 2, 3], [4, 5]]


In [7]:
e_nums, states, E_cas = solve_enums(H_cas, k, ne_per_block = ne_per_block,
                                    ne_range = ne_range, balance_t = balance_strength)
#     assert np.allclose(cas_tbt_tmp, cas_tbt), "changed"
print(f"e_nums:{e_nums}")
print(f"E_cas: {E_cas}")
tbt_2 = copy.deepcopy(cas_tbt)
killer_c, killer_obt, killer_tbt = construct_killer(k, e_nums, n = spatial_orbs, n_killer = n_killer)

Ne within current block: 4
E_min: -98.64331385684764 for orbs: [0, 1, 2, 3]
current state Energy: -98.64331385684734
Ne within current block: 3
E_min: -66.5845628678401 for orbs: [4, 5]
current state Energy: -66.58456286784005
e_nums:[4, 3]
E_cas: -165.2278767246874


In [8]:
if check_state:
    sd_sol = sdstate()
    for st in states:
        sd_sol = sd_sol.concatenate(st)
#     The following code segment checks the state energy for the full Hamiltonian, takes exponential space 
#     and time with respect to the number of blocks
    print(sd_sol.n_qubit)
    H_cas_op = feru.get_ferm_op(cas_tbt, False) + feru.get_ferm_op(cas_obt, False) 
    E_sol = sd_sol.exp(H_cas_op)
    print(f"Double check ground state energy: {E_sol}")

# Checking ground state with FCI
# Warning: This takes exponential time and space to run
#     Checking H_cas symmetries
if check_symmetry:
    Sz = of.hamiltonians.sz_operator(spatial_orbs)
    S2 = of.hamiltonians.s_squared_operator(spatial_orbs)
    assert of.FermionOperator.zero() == of.normal_ordered(of.commutator(Sz, H_cas)), "Sz symmetry broken"
    assert of.FermionOperator.zero() == of.normal_ordered(of.commutator(S2, H_cas)), "S2 symmetry broken"

if FCI:
    E_min, sol = of.get_ground_state(of.get_sparse_operator(H_cas_op))
    print(f"FCI Energy: {E_min}")
    tmp_st = sdstate(n_qubit = spin_orbs * 2)
    for s in range(len(sol)):
        if sol[s] > np.finfo(np.float32).eps:
            tmp_st += sdstate(s, sol[s])
    #         print(bin(s))
    print(tmp_st.norm())
    tmp_st.normalize()
    print(tmp_st.exp(H_cas))

killer_c, killer_obt, killer_tbt = construct_killer(k, e_nums, n = spatial_orbs, n_killer = n_killer)
# if check_symmetry:
#     assert of.FermionOperator.zero() == of.normal_ordered(of.commutator(Sz, cas_killer)), "Killer broke Sz symmetry"
#     assert of.FermionOperator.zero() == of.normal_ordered(of.commutator(S2, cas_killer)), "S2 symmetry broken"
# 
# # Checking: if FCI of killer gives same result. Warning; takes exponential time 
# if FCI:
#     sparse_with_killer = of.get_sparse_operator(cas_killer + H_cas_op)
#     killer_Emin, killer_sol = of.get_ground_state(sparse_with_killer)
#     print(f"FCI Energy solution with killer: {killer_Emin}")
#     sd_Emin = sd_sol.exp(cas_tbt) + sd_sol.exp(cas_killer)
#     print(f"difference with CAS energy: {sd_Emin - killer_Emin}")
# 
# #     Checking: if killer does not change ground state
# if check_state:
#     killer_error = sd_sol.exp(cas_killer)
#     print(f"Solution Energy shift by killer: {killer_error}")
#     killer_E_sol = sd_sol.exp(H_cas_op + cas_killer)
#     print(f"Solution Energy with killer: {killer_E_sol}")

12
Double check ground state energy: -165.22787672468746


In [9]:
planted_sol = {}
# planted_sol["E_min"] = E_sol
planted_sol["e_nums"] = e_nums
# planted_sol["sol"] = sd_sol
planted_sol["killer"] = (killer_c, killer_obt, killer_tbt)
planted_sol["k"] = k
planted_sol["casnum"] = casnum
planted_sol["pnum"] = pnum
planted_sol["upnum"] = upnum
planted_sol["spatial_orbs"] = spatial_orbs
planted_sol["cas_x"] = cas_x
if check_state:
    planted_sol["solution"] = sd_sol
    
ps_path = "planted_solutions/"
f_name = fcidump_filename.split(".")[0] + ".pkl"
print(ps_path + f_name)

l = list(map(len, k))
l = list(map(str, l))
key = "-".join(l)
print(key)
if os.path.exists(ps_path + f_name):
    with open(ps_path + f_name, 'rb') as handle:
        dic = pickle.load(handle)
else:
    dic = {}

with open(ps_path + f_name, 'wb') as handle:
    dic[key] = planted_sol
    pickle.dump(dic, handle, protocol=pickle.HIGHEST_PROTOCOL)

planted_solutions/fcidump.pkl
4-2


In [10]:
def get_cas_matrix(cas_x, n, k):
    obt = np.zeros([n, n])
    tbt = np.zeros([n, n, n, n])
    idx = 0
    for orbs in k:
        for p, q in product(orbs, repeat = 2):
            obt[p,q] = cas_x[idx]
            idx += 1
        for p, q, r, s in product(orbs, repeat = 4):
            tbt[p,q,r,s] = cas_x[idx]
            idx += 1
    return obt, tbt

def orbtransf(tensor, U, complex = False):
    """Return applying UHU* for the tensor representing the 1e or 2e tensor"""
    if len(tensor.shape) == 4:
        p = np.einsum_path('ak,bl,cm,dn,klmn->abcd', U, U, U, U, tensor)[0]
        return np.einsum('ak,bl,cm,dn,klmn->abcd', U, U, U, U, tensor, optimize = p)
    elif len(tensor.shape) == 2:
        p = np.einsum_path('ap,bq, pq->ab', U, U, tensor)[0]
        return np.einsum('ap,bq, pq->ab', U, U, tensor, optimize = p)

In [11]:
original, original_hidden, killer_added, killer_added_hidden = construct_Hamiltonian_with_solution(
    path="planted_solutions/", file_name="2_co2_6-311++G__.pkl"
)

In [ ]:
original_c, original_obt, original_tbt = original
original_hidden_c, original_hidden_obt, original_hidden_tbt = original_hidden
killer_added_c, killer_added_obt, killer_added_tbt = killer_added
killer_added_hidden_c, killer_added_hidden_obt, killer_added_hidden_tbt = killer_added_hidden

In [ ]:
if FCI:
    obt_to_use, tbt_to_use = chem_to_physicist_notation_spin_orbitals(tbt_2)
    num_spin_orbitals = tbt_to_use.shape[0]
    num_orbitals = num_spin_orbitals // 2
    num_electrons = 8
    num_unpaired_electrons = 0
    # multiplicity = num_unpaired_electrons + 1
    nuc_rep_energy=0
    init_state_bond_dimension= 100
    max_num_sweeps=20

    sweep_schedule_bond_dims = [init_state_bond_dimension] * 4 + [
        init_state_bond_dimension
    ] * 4
    sweep_schedule_noise = [1e-4] * 4 + [1e-5] * 4 + [0]
    sweep_schedule_davidson_threshold = [1e-10] * 8
    energy_convergence_threshold = 1e-8

    dmrg_parameters = {
                "factor_half_convention": True,
                "symmetry_type": "SGF",
                "num_threads": 1,
                "n_mkl_threads": 1,
                "num_orbitals": 2*num_orbitals, # Extra 2 necessary for SGF
                "num_spin_orbitals": 2*num_spin_orbitals, # Extra 2 necessary for SGF
                "num_electrons": num_electrons,
                "two_S": num_unpaired_electrons,
                "two_Sz": num_unpaired_electrons,
                "orb_sym": np.zeros(num_orbitals,dtype=np.int_).tolist(),
                "temp_dir": "./tests/temp",
                "stack_mem": 1073741824,
                "restart_dir": "./tests/restart",
                "core_energy": nuc_rep_energy,
                "reordering_method": "none",
                "init_state_seed": 64241,  # 0 means random seed
                "initial_mps_method": "random",
                "init_state_bond_dimension": init_state_bond_dimension,
                "occupancy_hint": None,
                "full_fci_space_bool": True,
                "init_state_direct_two_site_construction_bool": False,
                "max_num_sweeps": max_num_sweeps,
                "energy_convergence_threshold": energy_convergence_threshold,
                "sweep_schedule_bond_dims": sweep_schedule_bond_dims,
                "sweep_schedule_noise": sweep_schedule_noise,
                "sweep_schedule_davidson_threshold": sweep_schedule_davidson_threshold,
                "davidson_type": None,  # Default is None, for "Normal"
                "eigenvalue_cutoff": 1e-20,  # Cutoff of eigenvalues, default is 1e-20
                "davidson_max_iterations": 4000,  # Default is 4000
                "davidson_max_krylov_subspace_size": 50,  # Default is 50
                "lowmem_noise_bool": False,  # Whether to use a lower memory version of the noise, default is False
                "sweep_start": 0,  # Default is 0, where to start sweep
                "initial_sweep_direction": None,  # Default is None, True means forward sweep (left-to-right)
                "stack_mem_ratio": 0.8,  # Default is 0.4
            }
    dmrg_results_dict = dmrghandler.qchem_dmrg_calc.single_qchem_dmrg_calc(
        one_body_tensor=obt_to_use,
        two_body_tensor=tbt_to_use,
        dmrg_parameters=dmrg_parameters,
        verbosity=1,
    )

    dmrg_ground_state_energy = dmrg_results_dict["dmrg_ground_state_energy"]

In [ ]:
tbt_1_H_ij, tbt_1_G_ijkl = chem_to_physicist_notation_spin_orbitals(original_obt, original_tbt)

In [ ]:
tbt_3_hidden_H_ij, tbt_3_hidden_G_ijkl = chem_spin_orb_to_phys_spatial_orb(tbt_3_hidden)

In [ ]:
# Make dir
fcidump_output_path = Path(fcidump_output_path)
fcidump_output_path.mkdir(parents=True, exist_ok=True)

In [ ]:
label = "_tbt_1"
filename = fcidump_filename + f"{label}"
pyscf.tools.fcidump.from_integrals(
    fcidump_output_path/Path(filename),
    tbt_1_H_ij,
    tbt_1_G_ijkl,
    nmo=tbt_1_G_ijkl.shape[0],
    nelec=np.sum(e_nums),
    nuc=0,
    ms=0,
    orbsym=None,
    tol=1E-8,
    float_format=' %.16g',
)

In [ ]:
label = "_tbt_1_hidden"
filename = fcidump_filename + f"{label}"
pyscf.tools.fcidump.from_integrals(
    fcidump_output_path/Path(filename),
    tbt_1_hidden_H_ij,
    tbt_1_hidden_G_ijkl,
    nmo=tbt_1_G_ijkl.shape[0],
    nelec=np.sum(e_nums),
    nuc=0,
    ms=0,
    orbsym=None,
    tol=1E-8,
    float_format=' %.16g',
)